In [1]:
# Import the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Part 1 - Data Preprocessing

In [2]:
# Import the dataset
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:, 3:13].values
y = dataset.iloc[:, 13].values

# Encode categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_X_1 = LabelEncoder()
X[:, 1] = labelencoder_X_1.fit_transform(X[:, 1])
labelencoder_X_2 = LabelEncoder()
X[:, 2] = labelencoder_X_2.fit_transform(X[:, 2])
onehotencoder = OneHotEncoder(categorical_features = [1])
X = onehotencoder.fit_transform(X).toarray()
X = X[:, 1:]

# Split the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Part 2 - Making the ANN

In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense

# Three layers:
# 2 hidden layers - 6 units, relu
# 1 output layer - 1 unit, sigmoid
classifier = Sequential()

classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu', input_dim = 11))
classifier.add(Dense(units = 6, kernel_initializer = 'uniform', activation = 'relu'))
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))

# Binary cross-entropy for binary output
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.fit(X_train, y_train, batch_size = 10, epochs = 100)

Using TensorFlow backend.


Epoch 1/100
8000/8000 [==============================] - 1s 172us/step - loss: 0.4885 - acc: 0.7972 0s - loss: 0.5073 - acc: 0.7
Epoch 2/100
8000/8000 [==============================] - 1s 79us/step - loss: 0.4215 - acc: 0.8214
Epoch 3/100
8000/8000 [==============================] - 1s 76us/step - loss: 0.4160 - acc: 0.8300
Epoch 4/100
8000/8000 [==============================] - 1s 79us/step - loss: 0.4114 - acc: 0.8319
Epoch 5/100
8000/8000 [==============================] - 1s 76us/step - loss: 0.4079 - acc: 0.8342
Epoch 6/100
8000/8000 [==============================] - 1s 77us/step - loss: 0.4059 - acc: 0.8316
Epoch 7/100
8000/8000 [==============================] - 1s 78us/step - loss: 0.4047 - acc: 0.8335: 0s - loss: 0.4059 - acc: 0.8
Epoch 8/100
8000/8000 [==============================] - 1s 76us/step - loss: 0.4036 - acc: 0.8332
Epoch 9/100
8000/8000 [==============================] - 1s 79us/step - loss: 0.4029 - acc: 0.8340
Epoch 10/100
8000/8000 [=========================

# Part 3 - Making predictions and evaluating the model

In [15]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
import pandas as pd
pd.DataFrame(cm, index = ["Actual Positive", "Actual Negative"], columns = ["Predicted Postive", "Predicted Negative"])

,Predicted Postive,Predicted Negative
Actual Positive,1509,86
Actual Negative,202,203


# Part 4 - K-fold Cross validation
- note that Cross validation can be computationally heavy
- be sure to use cross_val_score( . . . , ***n_jobs = -1***) to use all CPUs when calculating

# Part 5 - Grid Search
- Iterate through a set of hyper-parameters

In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense

# Function to pass to grid search
def build_classifier(optimizer):
    classifier = Sequential()
    
    classifier.add(Dense(units=6, activation='relu', kernel_initializer = 'uniform', input_dim = 11))
    classifier.add(Dense(units=6, activation='relu',  kernel_initializer = 'uniform'))
    classifier.add(Dense(units=1, activation='sigmoid',  kernel_initializer = 'uniform'))
    
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

# Scikit-learn classifier AP implementation for Keras 
classifier = KerasClassifier(build_fn = build_classifier)

# Hyper-params
parameters = {'batch_size': [25, 32],
             'epochs': [100, 500],
             'optimizer': ['adam','rmsprop']}

grid_search = GridSearchCV(estimator = classifier, param_grid = parameters, cv = 10, n_jobs = -1)

# Train & retrive best results
grid_search = grid_search.fit(X_train, y_train)
best_params = gird_search.best_params_
best_accuracy = grid.search.best_score_

Using TensorFlow backend.


#### With dropout layers

In [36]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense, Dropout

# Function to pass to grid search
def build_classifier(optimizer, dropRate):
    classifier = Sequential()
    
    classifier.add(Dense(units=10, activation='relu', kernel_initializer = 'uniform', input_dim = 11))
    classifier.add(Dropout(dropRate))
    classifier.add(Dense(units=10, activation='relu',  kernel_initializer = 'uniform'))
    classifier.add(Dropout(dropRate))
    classifier.add(Dense(units=1, activation='sigmoid',  kernel_initializer = 'uniform'))
    
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

# Scikit-learn classifier AP implementation for Keras 
classifier = KerasClassifier(build_fn = build_classifier)

# Hyper-params
parameters = {'batch_size': [25, 32],
             'epochs': [100, 500],
             'optimizer': ['adam','rmsprop'],
             'dropRate': [0.1, 0.2, 0.3, 0.4, 0.5]}

grid_search = GridSearchCV(estimator = classifier, param_grid = parameters, cv = 10, n_jobs = -1)

# Train & retrive best results
grid_search = grid_search.fit(X_train, y_train)
best_params = gird_search.best_params_
best_accuracy = grid.search.best_score_

Epoch 1/1
7200/7200 [==============================] - 0s 35us/step
Epoch 1/1


KeyboardInterrupt: 

In [30]:
y_train.shape

(8000,)